# Staggered Grids

In this demo, we will look at how we can construct a staggered grid using some tools available within the `finitevolx` package.

We will go over some of the fundamental data containers when building a `Field` data structure.

* Domain
* Values

Then we will outline some of the fundamental operations that we can do when operating on fields.

* Selecting Axes
* Padding
* Mathematical Operations
* Difference
* Interpolation

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # second gpu
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'FALSE'

import jax
# again, this only works on startup!
from jax import config
config.update("jax_enable_x64", True)

In [2]:
import autoroot
import jax.numpy as jnp
from jaxtyping import Array

import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Domains

In [3]:
Nx, Ny = 200, 104
dx, dy = 5e3, 5e3
Lx, Ly = (Nx - 1) * dx, (Ny - 1) * dy

In [4]:
from somax.domain import Domain

xmin, ymin = 0.0, 0.0
xmax, ymax = Lx, Ly
h_domain = Domain(xmin=(xmin, ymin), xmax=(xmax, ymax), Lx=(Lx,Ly), Nx=(Nx,Ny), dx=(dx,dy))

## Field

In [6]:
from somax._src.field.field import Field

h = Field.init_from_ones(domain=h_domain)
h

Field(
  values=f64[200,104],
  domain=Domain(
    xmin=(0.0, 0.0),
    xmax=(995000.0, 515000.0),
    dx=(5000.0, 5000.0),
    Nx=(200, 104),
    Lx=(995000.0, 515000.0),
    ndim=2
  )
)

## Unary Operations

* Selecting Points
* Mathematical Operations
* Padding

### Subsetting

In [12]:
# subset
h_i = h[1:-1, :]


assert h_i.domain.xmin[0] == (xmin + dx)
assert h_i.domain.xmax[0] == (xmax - dx)

assert h_i.domain.Nx[0] == (Nx - 2)

assert h_i.domain.Lx[0] == (Lx - 2 * dx)

### Padding

In [13]:
from somax._src.operators.pad import pad_array, pad_domain, pad_field

#### Functional API

This is the functional API which operators on `Arrays` and `Domains` independently.

In [14]:
# pad axis (semantically the same)
pad_width = ("both", None)
pad_width = ((1,1), (0,0))

# pad the values array
h_pad: Array = pad_array(h.values, pad_width=pad_width)

# pad the domain
h_domain_pad: Domain = pad_domain(h_domain, pad_width=pad_width)

h_domain_pad

Domain(
  xmin=(-5000.0, 0.0),
  xmax=(1000000.0, 515000.0),
  dx=(5000.0, 5000.0),
  Nx=(202, 104),
  Lx=(1005000.0, 515000.0),
  ndim=2
)

In [15]:
assert h_domain_pad.xmin[0] == (xmin - dx)
assert h_domain_pad.xmax[0] == (xmax + dx)

assert h_domain_pad.Nx[0] == (Nx + 2)

assert h_domain_pad.Lx[0] == (Lx + 2 * dx)

#### Field API

This is the field API which operates on `Fields`. 
The underlying operations are exactly the same as the Functional API.
However, it offers the user a more convenient data container instead of having to deal with the `Array` and `Domain` independently.

In [16]:
# pad the field
h_pad: Field = pad_field(h, pad_width=pad_width)
h_pad

Field(
  values=f64[202,104],
  domain=Domain(
    xmin=(-5000.0, 0.0),
    xmax=(1000000.0, 515000.0),
    dx=(5000.0, 5000.0),
    Nx=(202, 104),
    Lx=(1005000.0, 515000.0),
    ndim=2
  )
)

In [17]:
assert h_pad.domain.xmin[0] == (xmin - dx)
assert h_pad.domain.xmax[0] == (xmax + dx)

assert h_pad.domain.Nx[0] == (Nx + 2)

assert h_pad.domain.Lx[0] == (Lx + 2 * dx)

## Staggered Domains

In [18]:
from somax._src.operators.stagger import stagger_domain

In this example, we'll stagger u to the **right**.

In [19]:

u_domain = stagger_domain(h_domain, direction=("right", None), stagger=(True, False))

# check xmin, xmax limits
assert u_domain.xmin[0] == h_domain.xmin[0] + 0.5 * dx
assert u_domain.xmax[0] == h_domain.xmax[0] + 0.5 * dx

# check Nx, Lx
assert u_domain.Nx[0] == h_domain.Nx[0]
assert u_domain.Lx[0] == h_domain.Lx[0]

In [20]:
dx

5000.0

In this example, we will stagger u to the right and left (so the outer).

In [21]:
u_domain = stagger_domain(h_domain, direction=("outer", None), stagger=(True, False))

# check xmin, xmax limits
assert u_domain.xmin[0] == h_domain.xmin[0] - 0.5 * dx
assert u_domain.xmax[0] == h_domain.xmax[0] + 0.5 * dx

# check Nx, Lx
assert u_domain.Nx[0] == h_domain.Nx[0] + 1
assert u_domain.Lx[0] == h_domain.Lx[0] + dx

In [22]:
u_domain = stagger_domain(h_domain, direction=("outer", None), stagger=(True, False))
v_domain = stagger_domain(h_domain, direction=(None, "outer"), stagger=(False, True))
q_domain = stagger_domain(h_domain, direction=("outer", "outer"), stagger=(True, True))

## Binary Operators

### Subsetting

### Padding